In [20]:
!git clone https://github.com/zach401/acnportal.git
!pip install acnportal/.

fatal: destination path 'acnportal' already exists and is not an empty directory.


Processing c:\users\omi\documents\uni\diplomovka\github\code rep\diplomovka_code\acnportal\my_experiments\python_implementation\mpc\acnportal
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for acnportal: filename=acnportal-0.3.3-py3-none-any.whl size=120552 sha256=3e80941c49cd434e231c1bee0839072eee6b696dca8a6ebd370551d93dda5397
  Stored in directory: C:\Users\OMI\AppData\Local\Temp\pip-ephem-wheel-cache-sca5dvs2\wheels\97\34\16\0275fe18247ebd48f261c6fc979226250361f9c1ebae1946c6
Successfully built acnportal
  Attempting uninstall: acnportal
    Found existing installation: acnportal 0.3.3
    Uninstalling acnportal-0.3.3:
      Successfully uninstalled acnportal-0.3.3



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
from datetime import datetime
import pytz
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from copy import deepcopy

from acnportal import algorithms
from acnportal import acnsim
from mpc_sched import *
from mpc_optim import *


In [22]:




# -- Experiment Parameters ---------------------------------------------------------------------------------------------
timezone = pytz.timezone('America/Los_Angeles')
start = timezone.localize(datetime(2018, 9, 5))
end = timezone.localize(datetime(2018, 9, 6))
period = 5  # minute
voltage = 220  # volts
default_battery_power = 32 * voltage / 1000 # kW
site = 'caltech'

# -- Network -----------------------------------------------------------------------------------------------------------
cn = acnsim.sites.caltech_acn(basic_evse=True, voltage=voltage)

# -- Events ------------------------------------------------------------------------------------------------------------
API_KEY = 'DEMO_TOKEN'
events = acnsim.acndata_events.generate_events(API_KEY, site, start, end, period, voltage, default_battery_power)


# -- Scheduling Algorithm ----------------------------------------------------------------------------------------------
# sch = EarliestDeadlineFirstAlgo(increment=1)
sch2 = algorithms.SortedSchedulingAlgo(algorithms.least_laxity_first)

# objectives can be:
#  maximise energy
#  minimize costs/ maximise profit
# charge as quickly as possible
# smoothen charging rates
obj = Objective()
objective = [
    obj.maximize_charging_energy(obj),
    obj.minimize_customers_chaging_costs(obj),
    obj.charge_as_quickly_as_possible(self=obj),
    # smoothen_charging_rates
]
sch_mpc = MyMpcAlgorithm(objective=objective)


sim_mpc = acnsim.Simulator(deepcopy(cn), sch2, deepcopy(events),start, period=period, verbose=False)
sim_mpc.run()
a = 0

In [23]:
# For comparison we will also run the builtin earliest deadline first algorithm
# sim2 = acnsim.Simulator(deepcopy(cn), sch2, deepcopy(events), start, period=period, verbose=False)
# sim2.run()